In [4]:
import os
import pickle
import numpy as np
import matplotlib.path as mpath
import scipy.sparse as sp
from scipy.interpolate import griddata
from scipy.interpolate import LinearNDInterpolator, CloughTocher2DInterpolator, CubicSpline, interp1d, PchipInterpolator, RegularGridInterpolator
from scipy.spatial import Delaunay
from scipy.ndimage import gaussian_filter
from spektral.data import Dataset, Graph
from spektral.data.loaders import DisjointLoader, SingleLoader
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Dropout, Dense
from tensorflow.keras.regularizers import l2
from spektral.layers import MessagePassing, GCNConv, GATConv, ECCConv
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
from codebase.gnn_functions import *

In [ ]:
sigma = 20
encoder_data_history_range = (-40,0+1,10)
file_path = r'datasets/4000.am'

slices = [slice(0,1001,1),slice(0,250,1),slice(250,500,1)]
data_train = loadData(file_path, np.arange(*encoder_data_history_range), gradsBC=True, slices=slices, sigma=sigma)
meshing_size = 0.1
data_train_remesh = [RemeshData(data_train[i],meshing_size) for i in range(0,len(data_train),30)]
slices_test = [slice(0,1001,1),slice(250,500,1),slice(250,500,1)]
data_test = loadData(file_path, np.arange(*encoder_data_history_range), gradsBC=True, slices=slices_test, sigma=sigma)
data_test_remesh = [RemeshData(data_test[i],meshing_size) for i in range(0,len(data_test),100)]

In [3]:
# Generate graph datasets:
dataset_train = FEMDataset(data_train_remesh,cache_file="data/cfd_dynamic_train_graphs_setup1.pkl",force_regenerate=True)
dataset_val = FEMDataset(data_test_remesh,cache_file="data/cfd_dynamic_test_graphs_setup1.pkl",force_regenerate=True)

# Normalize graph datasets:
x_mean, x_std, e_mean, e_std = normalize_dataset_features(dataset_train, dataset_val)

Created cache directory: data
Generating 33 new graph samples...
Caching 33 graphs to data/cfd_dynamic_train_graphs_setup1.pkl
Generating 10 new graph samples...
Caching 10 graphs to data/cfd_dynamic_test_graphs_setup1.pkl
Starting feature normalization...
Node feature global mean (from train_dataset, shape (23,)): [-0.18464912  0.35898146  0.15273103 -0.28329974 -0.22453131  0.3793628
  0.16170157 -0.29028523 -0.25424328  0.3951998   0.16979711 -0.29773837
 -0.27259547  0.41457883  0.17801748 -0.30564582 -0.30557045  0.42995834
  0.18568549 -0.3140693  -0.01706287 -0.00508426  0.2777778 ]
Node feature global std (from train_dataset, shape (23,)): [2.1612613  1.5076146  0.6517483  0.7138889  2.1600606  1.4946195
 0.6546287  0.71063995 2.1623158  1.4915944  0.657476   0.70789194
 2.1617823  1.472611   0.6601113  0.70593685 2.15482    1.4552897
 0.6623511  0.70429516 0.5454589  0.54432905 0.44789693]
Training node features normalized.
Validation node features normalized.
Edge feature glo

In [ ]:
import tensorflow as tf
import numpy as np
import scipy.sparse

# --- Determine Fixed Feature Dimensions ---
sample_graph = dataset_train[0]
F_x_dim = sample_graph.x.shape[1]
F_e_dim = sample_graph.e.shape[1]
F_y_dim = sample_graph.y.shape[1]
D_nodes_dim = sample_graph.fem_nodes.shape[1]
nodes_per_element_dim = sample_graph.fem_elements.shape[1]

# --- Define Input Signatures ---
common_input_signature = [
    tf.TensorSpec(shape=[None, F_x_dim], dtype=tf.float32, name="x"),
    tf.SparseTensorSpec(shape=[None, None], dtype=tf.float32),
    tf.TensorSpec(shape=[None, F_e_dim], dtype=tf.float32, name="e"), # Shape [0, F_e_dim] if no edges/features
    tf.TensorSpec(shape=[None, F_y_dim], dtype=tf.float32, name="y"),
    {
        "nodes": tf.TensorSpec(shape=[None, D_nodes_dim], dtype=tf.float32, name="mesh_nodes"),
        "elements": tf.TensorSpec(shape=[None, nodes_per_element_dim], dtype=tf.int32, name="mesh_elements"),
        "boundaryNodes": tf.TensorSpec(shape=[None], dtype=tf.int32, name="mesh_boundaryNodes"),
        "internalNodes": tf.TensorSpec(shape=[None], dtype=tf.int32, name="mesh_internalNodes"),
        "edge_vector": tf.TensorSpec(shape=[None, 2], dtype=tf.float32, name="mesh_edge_vector"),
        "edge_length": tf.TensorSpec(shape=[None, 1], dtype=tf.float32, name="mesh_edge_length"),
    }
]

# --- Create Model, Loss, Optimizer ---
loss_fn = tf.keras.losses.MeanSquaredError()
optimizer = tf.keras.optimizers.Adam(learning_rate=0.3e-3) # Use learning_rate
n_dims = 64
model = FEMGNN(hidden_dim=n_dims, n_gnn_layers=4, out_dim=F_y_dim, ecc_conv_kernel_network=[n_dims,n_dims])
n_epochs = 600
history = {'loss': [], 'val_loss': []}


# --- tf.function decorated training step ---
@tf.function(input_signature=common_input_signature)
def train_step(x_in, a_in, e_in, y_in, mesh_in):
    with tf.GradientTape() as tape:
        # Note: The model receives the dictionary directly as its first argument.
        # Ensure FEMGNN's call method expects: model_inputs, training=True
        # where model_inputs = {"x": x_in, "a": a_in, "e": e_in, "mesh": mesh_in}
        predictions = model({"x": x_in, "a": a_in, "e": e_in, "mesh": mesh_in}, training=True)
        loss = loss_fn(y_in, predictions)
    
    gradients = tape.gradient(loss, model.trainable_variables)
    optimizer.apply_gradients(zip(gradients, model.trainable_variables))
    return loss

# --- tf.function decorated evaluation step ---
@tf.function(input_signature=common_input_signature)
def evaluate_step(x_in, a_in, e_in, y_in, mesh_in):
    predictions = model({"x": x_in, "a": a_in, "e": e_in, "mesh": mesh_in}, training=False)
    loss = loss_fn(y_in, predictions)
    return loss

# --- Helper function to convert graph object to tensors ---
def prepare_graph_inputs(g_object, expected_F_e_dim):
    x_tensor = tf.convert_to_tensor(g_object.x, dtype=tf.float32)
    y_tensor = tf.convert_to_tensor(g_object.y, dtype=tf.float32)
    e_tensor = tf.convert_to_tensor(g_object.e, dtype=tf.float32)

    # Handle adjacency matrix
    A_csr = g_object.a.tocsr()
    A_coo = A_csr.tocoo()
    indices = np.vstack((A_coo.row, A_coo.col)).T
    a_tf_sparse = tf.SparseTensor(indices=indices, values=A_coo.data, dense_shape=A_coo.shape)
    a_tf_sparse = tf.sparse.reorder(a_tf_sparse)

    mesh_dict_tf = {
        "nodes": tf.convert_to_tensor(g_object.fem_nodes, tf.float32),
        "elements": tf.convert_to_tensor(g_object.fem_elements, tf.int32),
        "boundaryNodes": tf.convert_to_tensor(g_object.fem_boundaryNodes, tf.int32),
        "internalNodes": tf.convert_to_tensor(g_object.fem_internalNodes, tf.int32),
        "edge_vector": tf.convert_to_tensor(g_object.fem_edge_vector, tf.float32),
        "edge_length": tf.convert_to_tensor(g_object.fem_edge_length, tf.float32),
    }
    return x_tensor, a_tf_sparse, e_tensor, y_tensor, mesh_dict_tf

# --- Training Loop ---
for epoch in range(1, n_epochs + 1):
    epoch_loss_sum = 0.0
    np.random.shuffle(dataset_train) # Shuffle the actual list of graph objects
    dataset_train_sub = dataset_train[::] # Or however you subset

    for i, g in enumerate(dataset_train_sub):
        x_g, a_g, e_g, y_g, mesh_g = prepare_graph_inputs(g, F_e_dim)
        loss_val = train_step(x_g, a_g, e_g, y_g, mesh_g)
        epoch_loss_sum += loss_val.numpy() # Add .numpy() to get Python float
        current_avg_loss = epoch_loss_sum / (i + 1)
        print(f"\rEpoch {epoch}/{n_epochs};  Iteration = {i+1}/{len(dataset_train_sub)};  Loss = {current_avg_loss:.4g}", end='')
    
    avg_epoch_loss = epoch_loss_sum / len(dataset_train_sub)
    history['loss'].append(avg_epoch_loss)

    val_loss_sum = 0.0
    for g_val in dataset_val:
        x_v, a_v, e_v, y_v, mesh_v = prepare_graph_inputs(g_val, F_e_dim)
        loss_v = evaluate_step(x_v, a_v, e_v, y_v, mesh_v)
        val_loss_sum += loss_v.numpy()
    
    avg_val_loss = val_loss_sum / len(dataset_val)
    history['val_loss'].append(avg_val_loss)
    print(f";  Val Loss = {avg_val_loss:.4g}")

# --- Plot Training History ---s
plt.plot(history['loss'],label='loss')
plt.plot(history['val_loss'],label='val_loss')
plt.gca().set_yscale('log')
plt.legend()

/var/folders/d7/0s3_ltgs6476j7ypg4tch4xr0000gn/T/ipykernel_80709/3219315233.py:85: UserWarning: you are shuffling a 'FEMDataset' object which is not a subclass of 'Sequence'; `shuffle` is not guaranteed to behave correctly. E.g., non-numpy array/tensor objects with view semantics may contain duplicates after shuffling.
  np.random.shuffle(dataset_train) # Shuffle the actual list of graph objects


In [ ]:
data_eval_remesh = [RemeshData(data_test[i],meshing_size) for i in range(0,len(data_test),5)]
dataset_eval = FEMDataset(data_eval_remesh,cache_file="data/cfd_dynamic_test_graphs_setup1.pkl",force_regenerate=True)
dataset_train = FEMDataset(data_train_remesh,cache_file="data/cfd_dynamic_train_graphs_setup1.pkl",force_regenerate=True)
x_mean, x_std, e_mean, e_std = normalize_dataset_features(dataset_train, dataset_eval)

val_loss_sum = 0.0
eval_results = []
for g_eval in dataset_eval:
    x_v, a_v, e_v, y_v, mesh_v = prepare_graph_inputs(g_eval, F_e_dim)
    predictions = model({"x": x_v, "a": a_v, "e": e_v, "mesh": mesh_v}, training=False)
    loss = loss_fn(y_v, predictions)
    eval_results.append({
        'mesh': mesh_v,
        'predictions': predictions.numpy(),
        'ground_truth': y_v.numpy(),
        'loss': loss.numpy(),
        'err': predictions.numpy()-y_v.numpy(),
    })

err = np.array([d['loss'] for d in eval_results])
plt.plot(err)
plt.xlabels('timestep id')
plt.ylabels('MSE')